<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/Landsat8_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=72g4AgHwHUgtx4HxRm38QrnBrYMMJBLi6cl4HnC-kMw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi40Q6mat_dWsHNH9OAsZUfZJqL7vgWGkzbzzPrrm_hpuZ3xFxMMoI

Successfully saved authorization token.


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
wdpa_polygon = ee.FeatureCollection('users/minervas_jojo_code/KHM_polygons')
##https://code.earthengine.google.com/?asset=users/minervas_jojo_code/KHM_polygons


# select the pa in cambodia
khm = wdpa_polygon.filter(ee.Filter.eq('ISO3', 'KHM'))

In [ ]:
names = khm.aggregate_array("NAME").getInfo()
print(names)

['Phnom Samkos', 'Samlaut', 'Roniem Daun sam III', 'Roniem Daun Sam II', 'Tonle Sap Biosphere', 'Central Cardamom Mountains', 'Southwest Elephant Corridor', 'Botum Sakor', 'Dong Peng', 'Koh Kapik and Associated Islets', 'Peam Krasop', 'Kbal Chay', 'Ream', 'Kep', 'Preah Monivong (Bokor)', 'Phnom Aural', 'Kirirom', 'Ta Moa', 'Trapeang Lpeou', 'Snoul', 'Seima', 'Phnom Prich', 'Beng Per', 'Boeng Tonle Chhmar', 'Boeng Chhmar and Associated River System and Floodplain', 'Stung Sen', 'Prek Toal Ramsar Site', 'Prek Toal', 'Angkor Wat', 'Phnom Kulen', 'Lour Kola Peach Borant Angkor', 'Kulen Promtep', 'Preah Vihear Temple', 'Preah Vihear', 'Lomphat', 'Middle Stretches of the Mekong River north of Stoeng Treng', 'Virachey', 'Banteay Chhmar', 'Ang Trapeng Thmor', 'Roniem Daun Sam I', 'Mondulkiri', 'Nsok', 'Pnom Namlear', 'Snoul']


In [ ]:
aoi = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Phnom Kulen'))

In [ ]:

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
#image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]')
image=ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2016-01-01', '2016-12-31').median().select(bands))
landsat=image.clip(aoi)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

map4 = geemap.Map(center=[12.5657, 105],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(landsat, vis_params, 'Landsat-8')

map4.addLayerControl()
map4



Map(center=[12.5657, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [ ]:
# Load training points. The numeric property 'class' stores known labels.
#train = ee.FeatureCollection('users/minervas_jojo_code/train1').remap([1, 2,3,4,5,6,7], [0,1,2,3,4,5,6], 'class')
#test = ee.FeatureCollection('users/minervas_jojo_code/test1').remap([1, 2,3,4,5,6,7], [0,1,2,3,4,5,6], 'class');

In [ ]:
train=ee.FeatureCollection('users/minervas_jojo_code/train1').remap([1,2,3,4,5,6,7],[0,1,2,3,4,5,6],'class')
test=ee.FeatureCollection('users/minervas_jojo_code/test1').remap([1,2,3,4,5,6,7],[0,1,2,3,4,5,6],'class')

In [ ]:
# Overlay the points on the imagery to get training.
training = image.sampleRegions(
  collection=train,
  properties=['class'],
  scale=30)


random forest- training

In [ ]:
# Make a Random Forest classifier and train it.

#classifier=ee.Classifier.smileRandomForest(10).train(training,'class',bands)
classifier=ee.Classifier.smileRandomForest(10).train(training,'class',bands)

In [ ]:
#trainAccuracy=classifier.confusionMatrix()
trainAccuracy=classifier.confusionMatrix()
#print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Resubstitution error matrix: ',trainAccuracy.getInfo())
#print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())
print('Training overall accuracy: ',trainAccuracy.accuracy().getInfo())

Resubstitution error matrix:  [[45, 0, 1, 0, 0, 0, 0], [0, 19, 0, 0, 0, 0, 1], [0, 0, 44, 0, 0, 2, 0], [0, 1, 1, 43, 0, 0, 0], [0, 0, 3, 1, 26, 0, 0], [0, 0, 0, 1, 0, 52, 0], [0, 0, 0, 1, 3, 2, 33]]
Training overall accuracy:  0.9390681003584229


validation

In [ ]:
# Overlay the points on the imagery to get training.
testing = image.sampleRegions(
  collection=test,
  properties=['class'],
  scale=30)


In [ ]:
validated = testing.classify(classifier)

In [ ]:
validated.first().getInfo()

{'geometry': None,
 'id': '00002287a0e75a1979d1_0',
 'properties': {'B10': 2893,
  'B11': 2859,
  'B2': 337,
  'B3': 702,
  'B4': 634,
  'B5': 3442,
  'B6': 2319,
  'B7': 1253,
  'class': 3,
  'classification': 3},
 'type': 'Feature'}

In [ ]:
test_accuracy = validated.errorMatrix('class', 'classification') #actual lulc, classified lulc
test_accuracy.getInfo()

[[13, 0, 0, 0, 0, 0, 0],
 [0, 6, 0, 0, 0, 0, 0],
 [1, 0, 7, 0, 0, 0, 0],
 [0, 0, 0, 18, 0, 0, 0],
 [0, 0, 0, 0, 8, 0, 0],
 [0, 0, 0, 0, 0, 8, 0],
 [0, 0, 0, 0, 0, 0, 8]]

In [ ]:
test_accuracy.accuracy().getInfo()

0.9855072463768116

In [ ]:
test_accuracy.kappa().getInfo()
test_accuracy.producersAccuracy().getInfo()
test_accuracy.consumersAccuracy().getInfo()

[[0.9285714285714286, 1, 1, 1, 1, 1, 1]]